Przygotowanie danych

In [ ]:
import os

train_path = '/content/drive/MyDrive/Colab Notebooks/gesty/train'
test_path = '/content/drive/MyDrive/Colab Notebooks/gesty/test'

print("Katalog train:", os.listdir(train_path))
print("Katalog test:", os.listdir(test_path))


Katalog train: ['gun', 'devil', 'paper', 'rock', 'scissors', 'snake', 'sun']
Katalog test: ['devil', 'gun', 'paper', 'rock', 'scissors', 'snake', 'sun']


Import modulow

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

Implementacja

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

data = []
labels = []
classes = os.listdir(train_path)

for label_idx, class_name in enumerate(classes):
    class_dir = os.path.join(train_path, class_name)
    if not os.path.isdir(class_dir):
        continue

    print(f"Klasa {class_name}")

    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y])

                data.append(landmarks)
                labels.append(class_name)

print(f"Ilosc zebranych probek: {len(data)}")

Klasa gun
Klasa devil
Klasa paper
Klasa rock
Klasa scissors
Klasa snake
Klasa sun
Ilosc zebranych probek: 1620


In [ ]:
if len(data) > 0:
    X = np.array(data)
    y = np.array(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    svm = SVC(kernel='rbf', probability=True)
    svm.fit(X_train, y_train)

    y_pred = svm.predict(X_test)
    print("Dokładność:", accuracy_score(y_test, y_pred))
    print("\nRaport:\n", classification_report(y_test, y_pred))

    model_data = {
        'svm': svm,
        'classes': classes
    }

    with open('/content/drive/MyDrive/model_gesty.pkl', 'wb') as f:
        pickle.dump(model_data, f)

    print("Zapisano model")
else:
    print("Nie wykryto dloni na danych treningowych ")

hands.close()

Dokładność: 0.9783950617283951

Raport:
               precision    recall  f1-score   support

       devil       1.00      0.96      0.98        45
         gun       1.00      0.98      0.99        44
       paper       0.98      1.00      0.99        49
        rock       0.89      1.00      0.94        49
    scissors       1.00      1.00      1.00        42
       snake       1.00      0.94      0.97        47
         sun       1.00      0.98      0.99        48

    accuracy                           0.98       324
   macro avg       0.98      0.98      0.98       324
weighted avg       0.98      0.98      0.98       324

Zapisano model
